### OD-2. Оценка продуктовой фичи 

*Работу выполнила: Машковцева Дарья*

Цели

* Посчитать долю пользователей, листавших ленту, Retention Rate и ARPPU до и после внедрения фичи.
* Сформировать выводы по анализу.

In [36]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

In [37]:
# Подсчитаем количество зарегистрировавшихся пользователей registered_users_count за май и июнь 2019 года в разрезе registration_date — дней регистрации.

def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count = pd.DataFrame(getRegisteredUsersCount())
registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

In [38]:
# Получим данные об активных пользователях из базы и подсчитаем по дням количество активных пользователей active_users_count 
# с указанием даты регистрации registration_date.

registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])
def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())
for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])

In [39]:
# Объединяем две таблицы

retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')

In [40]:
# Найдем RETENTION RATE

retention_table['retention_rate'] = retention_table['active_users_count'] / retention_table['registered_users_count']
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу

In [41]:
# Получим данные о доходах

def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())
for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])
revenue = revenue.rename(columns={'date':'activity_date'})

In [42]:
# Объединяем две таблицы

retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)

In [43]:
# Рассчитаем Retention Rate седьмого дня за май 2019

retention_may = retention_table_with_revenue[retention_table_with_revenue['registration_date']>="2019-05-01"]
retention_may = retention_may[retention_may['registration_date']<"2019-06-01"]
retention_may_7 = retention_may[retention_may['lifetime']==7]

# Рассчитываем Retention Rate за май 2019

print(((retention_may_7['active_users_count'].sum())*100 / retention_may_7['registered_users_count'].sum()))

12.933000957960866


In [44]:
# Рассчитаем Retention Rate седьмого дня за июнь 2019

retention_june = retention_table_with_revenue[retention_table_with_revenue['registration_date']>="2019-06-01"]
retention_june = retention_june[retention_june['registration_date']<"2019-07-01"]
retention_june_7 = retention_june[retention_june['lifetime']==7]

# Рассчитываем Retention Rate за июнь 2019

print(((retention_june_7['active_users_count'].sum())*100 / retention_june_7['registered_users_count'].sum()))

12.560867530346634


In [45]:
# Рассчитаем ARPPU за май 2019

print(retention_may['revenue'].sum() /retention_may['users_count_with_revenue'].sum())

0.30612459519478075


In [46]:
# Рассчитаем ARPPU за июнь 2019

print(retention_june['revenue'].sum() /retention_june['users_count_with_revenue'].sum())

0.3931939549251713


Выводы:
* Был рассчитан коэффициент удержания пользователя Retention Rate за седьмой день. За май 2019 составляет 12,93%, а за июнь - 12,56%. 
* Была рассчитана средняя выручка от активного пользователя ARPPU. В мае 2019 она составила - 0.31, в июне - 0.39. 

Показатели после внедрения фичи не уменьшились, поэтому можно считать фичу эффективной. 
Но чтобы более точно сказать, что влияет на повышение ARPPU, необходимо провести дополнительный анализ.